# NER and Sentiment

In this section we will work through applying basic sentiment analysis to our data using a pre-built *distilBERT* model from the **Flair** library. We will then use our organization labels captured through NER in the previous section to create a list of organizations with the highest and lowest average sentiment scores.

In [1]:
import pandas as pd
import flair

We initialize the English sentiment model `en-sentiment`:

In [2]:
model = flair.models.TextClassifier.load('en-sentiment')

For each sample there are a few steps we need to take to create the sentiment score. We need to tokenize the input text, make a prediction, extract the direction (*positive* or *negative*) and confidence (a score from 0 to 1). If this is new to you, we cover the Flair sentiment model in more depth in **TK insert link**.

The following function carries out each of these steps for a single extract:

In [27]:
def get_sentiment(text):
    # tokenize input text
    sentence = flair.data.Sentence(text)
    # make sentiment prediction
    model.predict(sentence)
    # extract sentiment direction and confidence (label and score) object
    sentiment = sentence.labels[0]
   
    return sentiment

We now need to load our previously processed dataframe (which includes the *organizations* column) and `apply` the `get_sentiment` function to the *selftext* column. These sentiment scores will then be stored in a new *sentiment* column.

In [28]:
# load data
df = pd.read_csv('./data/processed/reddit_investing_ner.csv', sep='|')
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations
0,t3_139yfid,1683398660,investing,Low-end primary residence vs index stock portf...,"This is quite anecdotal, and only applies to s...",1.00,3,0,3,['150k']
1,t3_139y0se,1683397773,investing,Loaning shares to short sellers?,So my understanding about short-selling is tha...,0.67,1,0,1,[]
2,t3_139xtc6,1683397318,investing,AI doesn’t beat human intelligence… Charlie Mu...,Asked ChatGBT what it thinks about the annual ...,0.32,0,0,0,"['YouTube', 'AI']"
3,t3_139wtm2,1683395147,investing,Taxes on dividend income vs salary?,I'm 31 and I make $115k USD pretax and my inve...,0.76,12,0,12,[]
4,t3_139wt3y,1683395120,investing,Would the demystification and increase in publ...,There are a couple of factors I'm thinking of:...,0.84,26,0,26,[]


In [29]:
# get sentiment
df['sentiment'] = df['selftext'].apply(get_sentiment)
# df['sentiment_value'] = df['sentiment'].apply(lambda x: x.value)
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations,sentiment
0,t3_139yfid,1683398660,investing,Low-end primary residence vs index stock portf...,"This is quite anecdotal, and only applies to s...",1.00,3,0,3,['150k'],"Sentence[356]: ""This is quite anecdotal, and o..."
1,t3_139y0se,1683397773,investing,Loaning shares to short sellers?,So my understanding about short-selling is tha...,0.67,1,0,1,[],"Sentence[103]: ""So my understanding about shor..."
2,t3_139xtc6,1683397318,investing,AI doesn’t beat human intelligence… Charlie Mu...,Asked ChatGBT what it thinks about the annual ...,0.32,0,0,0,"['YouTube', 'AI']","Sentence[119]: ""Asked ChatGBT what it thinks a..."
3,t3_139wtm2,1683395147,investing,Taxes on dividend income vs salary?,I'm 31 and I make $115k USD pretax and my inve...,0.76,12,0,12,[],"Sentence[123]: ""I'm 31 and I make $115k USD pr..."
4,t3_139wt3y,1683395120,investing,Would the demystification and increase in publ...,There are a couple of factors I'm thinking of:...,0.84,26,0,26,[],"Sentence[274]: ""There are a couple of factors ..."


Now we need to extract each of the organizations alongside it's sentiment score. We will then loop through each, tallying up a total sentiment score and count.

Before we do that, we need to convert each value in the *organizations* column to a list (they are currently strings because we cannot save Python lists to file within Pandas dataframes, they are automatically converted to strings).

In [30]:
import ast

df['organizations'] = df['organizations'].apply(lambda x: ast.literal_eval(x))

In [50]:
# initialize sentiment dictionary
sentiment = {}

# loop through dataframe and extract org labels and sentiment scores into sentiment dictionary
for i, row in df.iterrows():
    # extract sentiment direction and score
    direction = row['sentiment'].value
    score = row['sentiment'].score
    # loop through each label in organizations column
    for org in row['organizations']:
        # check if org label exists in sentiment dictionary already
        if org not in sentiment.keys():
            # if it doesn't, initialize new entry in dictionary
            sentiment[org] = {'POSITIVE': [], 'NEGATIVE': []}
        # append positive/negative score to respective dictionary entry
        sentiment[org][direction].append(score)

In [51]:
# sentiment

In [52]:
# for el in sentiment.keys():
#     print(el)

In [53]:
sentiment['AI']

{'POSITIVE': [0.7963181138038635,
  0.7719136476516724,
  0.9994841814041138,
  0.992745041847229,
  0.6425257325172424],
 'NEGATIVE': [0.9999847412109375,
  0.999944806098938,
  0.9681519865989685,
  0.9712378978729248,
  0.6264246702194214,
  0.9971345663070679,
  0.9999116659164429,
  0.9631993174552917,
  0.9999866485595703]}

Now we can loop through each organization entry in the sentiment dictionary and calculate an average positive, and average negative score:

In [54]:
# initialize sentiment list
avg_sentiment = []

# loop through each organization
for org in sentiment.keys():
    # get number of positive and negative ratings
    freq = len(sentiment[org]['POSITIVE']) + len(sentiment[org]['NEGATIVE'])
    for direction in ['POSITIVE', 'NEGATIVE']:
        # assign to variable for cleaner code
        score = sentiment[org][direction]
        # if there are no entries, set to 0
        if len(score) == 0:
            sentiment[org][direction] = 0.0
        else:
            # otherwise calculate total
            sentiment[org][direction] = sum(score)
    # now calculate total amount
    total = sentiment[org]['POSITIVE'] - sentiment[org]['NEGATIVE']
    # and the average score
    avg = total/freq
    # add to sentiment list
    avg_sentiment.append({
        'entity': org,
        'positive': sentiment[org]['POSITIVE'],
        'negative': sentiment[org]['NEGATIVE'],
        'frequency': freq,
        'score': avg
    })

In [55]:
sentiment_df = pd.DataFrame(avg_sentiment)
sentiment_df.head()

,entity,positive,negative,frequency,score
0,150k,0.000000,0.999295,1,-0.999295
1,YouTube,0.000000,0.999985,1,-0.999985
2,AI,4.202987,8.525976,14,-0.308785
3,WeBull,0.000000,0.997950,1,-0.997950
4,Interactive Brokers,0.000000,2.995758,3,-0.998586


Immediately we can see we have a lot of entities which have appeared once in our dataset, and because of this their score will be pushed to one extreme or the other. We can filter out anything with less than or equal to a frequency of `3` to remove many of these instances:

In [56]:
sentiment_df = sentiment_df[sentiment_df['frequency'] > 3]
sentiment_df

,entity,positive,negative,frequency,score
2,AI,4.202987,8.525976,14,-0.308785
6,IRS,0.000000,6.976503,7,-0.996643
7,Wiki,0.000000,46.935214,47,-0.998622
8,FAQ,0.000000,46.935214,47,-0.998622
13,USD,0.816391,16.271929,18,-0.858641
...,...,...,...,...,...
581,Silicon Valley Bank,0.000000,5.971497,6,-0.995250
586,PS,0.000000,3.973953,4,-0.993488
621,SEP,0.000000,3.895827,4,-0.973957
674,Block,0.992289,2.973263,4,-0.495243


Here we have some more relevant information. We can see a few items that we can remove through the `BLACKLIST` covered in earlier sections such as *Fed* and *Treasury*, but nonetheless this list is looking much better than before. We can apply `sort` to search for the entities with the highest overall score:

In [57]:
sentiment_df.sort_values('score', ascending=False).head(10)

,entity,positive,negative,frequency,score
84,UBS,1.940008,1.998257,4,-0.014562
307,Microsoft,1.794636,1.955573,4,-0.040234
2,AI,4.202987,8.525976,14,-0.308785
274,IPO,1.685790,3.998862,6,-0.385512
674,Block,0.992289,2.973263,4,-0.495243
117,Morgan Stanley,0.998318,2.990525,4,-0.498052
304,Amazon,1.793350,5.908100,8,-0.514344
333,ROTH,0.825375,2.936386,4,-0.527753
534,Credit Suisse,0.976180,3.712336,5,-0.547231
260,VOO,4.134472,17.159200,23,-0.566293


Very quickly we've got our results that we have pulled together using simple, ready-to-use models, and **zero** text preprocessing. With further fine-tuning, and process development, these already good results can become great. Which we will cover soon.